In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
file_path = "/content/drive/My Drive/Datasets/data/Mx_Min_Price.csv"
df=pd.read_csv(file_path)
df


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Datasets/data/Mx_Min_Price.csv'

In [ ]:
file_path = "/content/drive/My Drive/Datasets/data/Order_details.csv"
df1=pd.read_csv(file_path,encoding='latin1')
df1


In [ ]:
from sklearn.impute import SimpleImputer
numeric_cols = df1.select_dtypes(include=['float', 'int']).columns

# Apply mean imputation only to numeric columns
imputer = SimpleImputer(strategy='mean')
df1[numeric_cols] = imputer.fit_transform(df1[numeric_cols])


In [ ]:
file_path = "/content/drive/My Drive/Datasets/data/Property_details.csv"
df2=pd.read_csv(file_path,encoding='latin1')
df2

**Data cleaning**

In [ ]:
values_df1 = set(df1['sourceurl'].unique())
values_df2 = set(df2['url'].unique())

# Intersection
overlap = values_df1.intersection(values_df2)
print(f"Number of overlapping values: {len(overlap)}")
print(f"Percentage of df1 values in overlap: {len(overlap) / len(values_df1) * 100:.2f}%")
print(f"Percentage of df2 values in overlap: {len(overlap) / len(values_df2) * 100:.2f}%")


**FEATURES ENGINEERING**

In [ ]:
categorical_columns = df1.select_dtypes(include='object').columns.tolist()
categorical_columns

We will gather usefull information from these columns and generate new columns

In [ ]:
#Roomamenities
unique_amenities = set()
for amenities in df1['roomamenities']:
    if isinstance(amenities, str):
        for amenity in amenities.split(';'):
            amenity = amenity.strip()
            if amenity:
                unique_amenities.add(amenity)

print(unique_amenities)
len(unique_amenities)


In [ ]:
for amenity in unique_amenities:
    df1[amenity] = df1['roomamenities'].apply(lambda x: amenity in x if isinstance(x, str) else False)
df1

In [ ]:
df1.drop(columns=["roomamenities"],inplace=True)

In [ ]:
 df1["ratedescription"].to_list()


In [ ]:
#Extract Room size from ratedescription
import re
size_pattern = r'Room\s*size:\s*(\d+)\s*m[²²]?'

# Function to extract room size in square meters
def extract_size(description):
    if isinstance(description, str):
        size_match = re.search(size_pattern, description)
        if size_match:
            return int(size_match.group(1))
    return None

# Apply the function to extract room size for each description
df1['Roomsize'] = df1['ratedescription'].apply(extract_size)

In [ ]:
df1["Roomsize"]

In [ ]:
df1["Roomsize"].isnull().sum()

In [ ]:
df1.drop(columns=["ratedescription"],inplace=True)

In [ ]:
df1["ratetype"]

In [ ]:
#Extract Free Cancellation And Pay at the hotel columns from rate Type
df1['Free cancellation'] = df1['ratetype'].str.contains('Free cancellation', case=False, na=False).astype(int)
df1['Pay at the hotel'] = df1['ratetype'].str.contains('Pay at the hotel', case=False, na=False).astype(int)

In [ ]:
df1["Pay at the hotel"].value_counts()

In [ ]:
df1['Free cancellation'].value_counts()

In [ ]:
df1.drop(columns=["ratetype"],inplace=True)

In [ ]:
def standardize_room_type(room_type):
    room_type = room_type.strip()
    room_type = re.sub(r'\s+', ' ', room_type)  # Replace multiple spaces with a single space
    return room_type

df1['roomtype'] = df1['roomtype'].apply(standardize_room_type)


In [ ]:
df1["roomtype"]

In [ ]:
# Function to extract features
def extract_room_features(room_type):
    features = {
        'num_beds': 0,
        'Single': 0,
        'Double': 0,
        'Twin': 0,
        'Triple': 0,
        'Quadruple': 0,
        'Suite': 0,
        'Deluxe': 0,
        'Superior': 0,
        'Standard': 0,
        'Family': 0,
        'Sea_View': 0,
        'Balcony': 0,
        'Apartment': 0,
        'Studio': 0,
        'King': 0,
        'Queen': 0
    }

    # Extracting number of beds
    bed_match = re.search(r'(\d+)\s*(bed|beds)', room_type, re.IGNORECASE)
    if bed_match:
        features['num_beds'] = int(bed_match.group(1))

    # Extract specific terms
    if 'Single' in room_type:
        features['Single_Room'] = 1
    if 'Double' in room_type:
        features['Double_Room'] = 1
    if 'Twin' in room_type:
        features['Twin_Room'] = 1
    if 'Triple' in room_type:
        features['Triple_Room'] = 1
    if 'Quadruple' in room_type:
        features['Quadruple_Room'] = 1
    if 'Suite' in room_type:
        features['Suite_Room'] = 1
    if 'Deluxe' in room_type:
        features['Deluxe_Room'] = 1
    if 'Superior' in room_type:
        features['Superior_Room'] = 1
    if 'Standard' in room_type:
        features['Standard_Room'] = 1
    if 'Family' in room_type:
        features['Family_Room'] = 1
    if 'Sea View' in room_type:
        features['Sea_View'] = 1
    if 'Balcony' in room_type:
        features['Balcony'] = 1
    if 'Apartment' in room_type:
        features['Apartment'] = 1
    if 'Studio' in room_type:
        features['Studio_Room'] = 1
    if 'King' in room_type:
        features['King_Room'] = 1
    if 'Queen' in room_type:
        features['Queen_Room'] = 1

    return pd.Series(features)

features_df = df1['roomtype'].apply(extract_room_features)
df1 = pd.concat([df1, features_df], axis=1)


In [ ]:
df1.drop(columns=["roomtype"])

In [ ]:
def convert_booleans_to_integers(dfa):
    for col in dfa.select_dtypes(include='bool').columns:
        dfa[col] = dfa[col].astype(int)
    return dfa
convert_booleans_to_integers(df1)

In [ ]:
df1.drop(columns=["sourceurl"],inplace=True)

In [ ]:
#since ispromo is object and not bool we independtly convert it to discrete
df1["ispromo"]=df1["ispromo"].replace({"Y":1,"N":0})

In [ ]:
df1["ispromo"]

In [ ]:
df1["closed"]=df1["closed"].replace({"Y":1,"N":0})


In [ ]:
df1["closed"]

In [ ]:
df1["promoname"].unique()

In [ ]:
#Extract Discount percentage from promoname and one hot encode common phrases into binary columns
import re
def extract_discount(description):
  description=str(description)
  match = re.search(r'(\d+)% discount', description)
  return int(match.group(1)) if match else 0

# Apply the function to each element in the 'promoname' column
df1['discount_percent'] = df1['promoname'].apply(extract_discount)

df1["discount_percent"]

In [ ]:
# List of common phrases to encode
phrases = ['Early Booking Saver', 'Limited time offer', 'Super Hot Deal', 'Expiring soon', 'Book Now', '72 hr limited sale', 'Last Minute Special', 'Weekend Special', 'Super Saver']
df1['promoname'] = df1['promoname'].fillna('')
# Create binary columns for each phrase
for phrase in phrases:
    df1[phrase.replace(' ', '_').replace('.', '')] = df1['promoname'].str.contains(phrase).astype(int)


In [ ]:
df1.drop(columns=["promoname"],inplace=True)

In [ ]:
df1["proxyused"].value_counts()

In [ ]:
df1.drop(columns=["proxyused"],inplace=True)

In [ ]:
df1["mealinclusiontype"]

In [ ]:
import re
# Function to standardize text
def standardize_text(text):
    if isinstance(text, str):
        text = text.replace('Free breakfast for {', 'Free breakfast for ')
        text = text.replace('}', '')
    return text
df1['mealinclusiontype'] = df1['mealinclusiontype'].fillna('').apply(standardize_text)

def extract_features(text):
    features = {
        'num_people': 0,  # Default value if text is empty or no number is found
        'Free_Breakfast': 0,
        'Dinner': 0,
        'Lunch': 0,
        'Free_WiFi': 0,
        'Parking': 0,
        'All_Inclusive': 0,
        'Free_sauna_access': 0,
        'Free_Fitness_Center_Access': 0,
        'Beverages': 0
    }

    # Extract the number of people if the text is not empty
    if text:
        # Use case-insensitive search for number of people
        match = re.search(r'free breakfast for (\d+)', text, re.IGNORECASE)
        if match:
            num_people = int(match.group(1))
            features['num_people'] = num_people
            features['Free_Breakfast'] = num_people
        elif re.search(r'free breakfast', text, re.IGNORECASE):
            features['Free_Breakfast'] = 1

        # Use lowercase for other feature checks
        lower_text = text.lower()
        features['Dinner'] = 1 if 'dinner' in lower_text else 0
        features['Lunch'] = 1 if 'lunch' in lower_text else 0
        features['Free_WiFi'] = 1 if 'free wifi' in lower_text else 0
        features['Parking'] = 1 if 'parking' in lower_text else 0
        features['All_Inclusive'] = 1 if 'all inclusive' in lower_text else 0
        features['Free_sauna_access'] = 1 if 'free sauna access' in lower_text else 0
        features['Free_Fitness_Center_Access'] = 1 if 'free fitness center access' in lower_text else 0
        features['Beverages'] = 1 if 'beverages' in lower_text else 0

    return pd.Series(features)

# Apply feature extraction
features_df = df1['mealinclusiontype'].apply(extract_features)
df1 = pd.concat([df1, features_df], axis=1)




In [ ]:
df1["num_people"]

In [ ]:
df1["Free_Breakfast"].value_counts()

In [ ]:
df1.drop(columns=["mealinclusiontype"],inplace=True)

Generating Features from dates


In [ ]:
#There are three coulmns containing date.dtcollected,reservation date and input dtcollected

In [ ]:
df1["dtcollected"]=pd.to_datetime(df1["dtcollected"], infer_datetime_format=True)

In [ ]:
df1["reservation date"]=pd.to_datetime(df1["reservation date"], infer_datetime_format=True)

In [ ]:
df1["input_dtcollected"]=pd.to_datetime(df1["reservation date"], infer_datetime_format=True)

In [ ]:
df1['dtmonth'] = df1['dtcollected'].dt.month
df1['dtyear'] = df1['dtcollected'].dt.year
df1['dtday'] = df1['dtcollected'].dt.day
df1['reservation_month'] = df1['reservation date'].dt.month
df1['reservation_year'] = df1['reservation date'].dt.year
df1['reservation_day'] = df1['reservation date'].dt.day
df1['input_dtmonth'] = df1['input_dtcollected'].dt.month
df1['input_dtyear'] = df1['input_dtcollected'].dt.year
df1['input_dtday'] = df1['input_dtcollected'].dt.day

In [ ]:
df1.drop(columns=["dtcollected","reservation date","input_dtcollected"],inplace=True)

In [ ]:
categorical_columns = df1.select_dtypes(include='object').columns.tolist()
categorical_columns

**FEATURES SELECTION**

Identify and drop single-Value columns as they have zero variance and do not contribute to the predictive power of a model

In [ ]:
single_value_columns = [col for col in df1.columns if df1[col].nunique() == 1]
single_value_columns

In [ ]:
df1.drop(columns=single_value_columns,inplace=True)

In [ ]:
def select_bincolumns(dataframe):
  binary_cols=[]
  for column in dataframe.columns:
    if df1[column].nunique()==2:
      binary_cols.append(column)
  return binary_cols
bin_cols=select_bincolumns(df1)
bin_cols

In [ ]:
def select_multicatcol(dataframe,threshold):
  multi_cols=[]

  for column in dataframe.columns:
    unique_values = dataframe[column].nunique()
    total_values = len(dataframe[column])
    unique_ratio = unique_values / total_values
    if unique_ratio<threshold and unique_values>2:
      multi_cols.append(column)

  return multi_cols
multi_columns=select_multicatcol(df1,0.05)
multi_columns

In [ ]:
numerical_columns = df1.select_dtypes(include=['number']).columns.tolist()
numerical_columns

numerical_columns_excluding_binary_multiclass = [
    col for col in numerical_columns if col not in bin_cols and col not in multi_columns
]

In [ ]:
numerical_columns_excluding_binary_multiclass

In [ ]:
import scipy.stats as stats
def pearson_corr(a,b):
   corr, p_value = stats.pearsonr(a, b)
   return corr, p_value


In [ ]:
# Interpret the results
dropped_num_cols=[]
alpha = 0.05
for column in numerical_columns_excluding_binary_multiclass:
  corr,p_value=pearson_corr(df1[column],df1["onsiteprice"])

  if p_value < alpha:
      print("There is a significant correlation between variable1 and variable2.")
  else:
      print("There is no significant correlation between variable1 and variable2.")
      dropped_num_cols.append(column)
dropped_num_cols

In [ ]:
#Drop id column
df1.drop(columns=["id"],inplace=True)

If features are binary variable and target is continous then t-test is applied to determine if there is a relationship between the two variables


In [ ]:
def select_bincolumns(dataframe):
  binary_cols=[]
  for column in dataframe.columns:
    if df1[column].nunique()==2:
      binary_cols.append(column)
  return binary_cols
bin_cols=select_bincolumns(df1)
bin_cols

In [ ]:
from scipy.stats import ttest_ind
def perform_t_test(group1, group2, alpha=0.05):
    t_stat, p_value = ttest_ind(group1, group2)
    return t_stat, p_value



In [ ]:
alpha=0.05
t_test_results={}
dropped_columns=[]
for column in bin_cols:
  group1=df1[df1[column]==0]["onsiteprice"]
  group2=df1[df1[column]==1]["onsiteprice"]
  t_stat, p_value=perform_t_test(group1, group2)
  t_test_results[column] = {'t_statistic': t_stat, 'p_value': p_value}
  if p_value >= alpha:
    result = "Reject H0: There is a significant difference between means.Drop Feature"
    dropped_columns.append(column)
  else:
    result = "Fail to reject H0: No significant difference between means.Select Feature"

# Display results
for col, result in t_test_results.items():
    print(f"Column: {col}, T-statistic: {result['t_statistic']}, P-value: {result['p_value']}")
    print()



In [ ]:
dropped_columns

In [ ]:
#dropping these columns
df1.drop(columns=dropped_columns,inplace=True)

**If the target variable is continous and features are mulit-class categorical we use ANOVA Test to find wether or not there is a relationship**

In [ ]:
def select_multicatcol(dataframe,threshold):
  multi_cols=[]

  for column in dataframe.columns:
    unique_values = dataframe[column].nunique()
    total_values = len(dataframe[column])
    unique_ratio = unique_values / total_values
    if unique_ratio<threshold and unique_values>2:
      multi_cols.append(column)

  return multi_cols
multi_columns=select_multicatcol(df1,0.05)
multi_columns

In [ ]:
import pandas as pd
from scipy.stats import f_oneway

# Function to perform ANOVA
def perform_anova(dataframe, categorical_column, continuous_column, alpha=0.05):
    dataframe[categorical_column] = dataframe[categorical_column].astype('category')
    groups = [dataframe[dataframe[categorical_column] == category][continuous_column] for category in dataframe[categorical_column].cat.categories]
    f_statistic, p_value = f_oneway(*groups)
    # Decision rule
    if p_value < alpha:
        decision = "Keep Feature"
    else:
        decision = "Drop Feature"

    return {
        'F-statistic': f_statistic,
        'p-value': p_value,
        'decision': decision
    }

# Example DataFrame and variables
alpha = 0.05
anova_results = {}
continuous_column = "onsiteprice"  # Replace with your actual continuous target column name
columns_to_drop = []

# Loop through each multi-class categorical column
for column in multi_columns:  # Replace with your actual list of multi-class columns
    results = perform_anova(df1, column, continuous_column)
    anova_results[column] = results

    print(f"Column: {column}, F-statistic: {results['F-statistic']}, P-value: {results['p-value']}")
    print(f"Decision: {results['decision']}")
    print()
    # Append column to drop list if decision is to drop feature
    if results['decision'] == "Drop Feature":
        columns_to_drop.append(column)

# Display all ANOVA results
print("Columns to drop:", columns_to_drop)
for col, result in anova_results.items():
    print(f"Column: {col}, F-statistic: {result['F-statistic']}, P-value: {result['p-value']}")
    print()


In [ ]:
columns_to_drop

In [ ]:
df1.drop(columns=columns_to_drop,inplace=True)

In [ ]:
# dfa=df1.copy()

In [ ]:
# from sklearn.impute import KNNImputer
# new_df = dfa[["Roomsize"]]
# knn_imputer = KNNImputer(n_neighbors=5)  # n_neighbors is the number of neighboring samples to use
# imputed_array = knn_imputer.fit_transform(new_df)

# # Create a new DataFrame with the imputed values
# imputed_df = pd.DataFrame(imputed_array, columns=["Roomsize"])

# # Replace the original 'roomsize' column with the imputed values
# dfa['Roomsize'] = imputed_df['Roomsize']


In [ ]:
df1.columns.to_list()

In [ ]:
# from sklearn.impute import KNNImputer
# new_df = df1[["Roomsize"]]
# knn_imputer = KNNImputer(n_neighbors=5)  # n_neighbors is the number of neighboring samples to use
# imputed_array = knn_imputer.fit_transform(new_df)

# # Create a new DataFrame with the imputed values
# imputed_df = pd.DataFrame(imputed_array, columns=["Roomsize"])

# # Replace the original 'roomsize' column with the imputed values
# df1['Roomsize'] = imputed_df['Roomsize']

In [ ]:
df1["roomtype"]

In [ ]:
df1.drop(columns=["roomtype"])

In [ ]:
X=df1.drop(columns=["onsiteprice"])
y=df1["onsiteprice"]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
# import pandas as pd

# # Assuming your final DataFrame is df_final
# df1.to_csv('/content/drive/My Drive/Datasets/data/final_dataframe.csv', index=False)
